In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install simplet5

In [1]:
!pip install -q -U  datasets accelerate peft trl bitsandbytes

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.0.11 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is i

In [2]:
from datasets import load_dataset, Dataset, DatasetDict
dataset = load_dataset("mayank200456789/Viva")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 381
    })
})

In [3]:
def remove_blank_strings(dataset):
    return dataset.filter(lambda example: example['text'] != "")

# Apply the function to the 'train' split
dataset["train"] = remove_blank_strings(dataset["train"])

# Print the modified dataset
print(DatasetDict(dataset))

Filter:   0%|          | 0/381 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 221
    })
})


In [5]:
dataset = {
    "train": Dataset.from_dict({
        "instruction": dataset["train"]["text"][:220:2],
        "output": dataset["train"]["text"][1::2]
    })
}


print(DatasetDict(dataset))

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 110
    })
})


In [8]:
df = pd.DataFrame(dataset['train'])

# Export DataFrame to CSV
df.to_csv('dataset.csv', index=False)

In [9]:
df_train=df
df_train = df_train.rename(columns={"output":"target_text", "instruction":"source_text"})
df_train = df_train[['source_text', 'target_text']]
 
# T5 model expects a task related prefix: since it is a question answering task, we will add a prefix "answer: "
df_train['source_text'] = "answer: " + df_train['source_text']

In [ ]:
from simplet5 import SimpleT5
 
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")from simplet5 import SimpleT5
 
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

In [ ]:
df_train=df
df_train = df_train.rename(columns={"output":"target_text", "instruction":"source_text"})
df_train = df_train[['source_text', 'target_text']]
df_train['source_text'] = "answer: " + df_train['source_text']

In [ ]:
from simplet5 import SimpleT5
 
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=df_train,
            eval_df=df_train[89:91],
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=16, max_epochs=25, use_gpu=True)

In [ ]:
model.train(train_df=df_train,
            eval_df=df_train[80:82],
            source_max_token_len=128,
            target_max_token_len=50,
            batch_size=16, max_epochs=3, use_gpu=True,outputdir = 'trained_t5')

In [ ]:
import os

def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

folder_path = '/kaggle/working'
remove_folder_contents(folder_path)
os.rmdir(folder_path)